In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools

In [2]:
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

In [3]:
lines_filepath = os.path.join("cornell movie-dialogs corpus", "movie_lines.txt")
conv_filepath = os.path.join("cornell movie-dialogs corpus", "movie_conversations.txt")

In [4]:
with open (lines_filepath, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines[:8]:
        print(line.strip()) 

L1045	u0	m0	BIANCA	They do not!
L1044	u2	m0	CAMERON	They do to!
L985	u0	m0	BIANCA	I hope so.
L984	u2	m0	CAMERON	She okay?
L925	u0	m0	BIANCA	Let's go.
L924	u2	m0	CAMERON	Wow
L872	u0	m0	BIANCA	Okay -- you're gonna need to learn how to lie.
L871	u2	m0	CAMERON	No


In [5]:
line_fields = ["lineID", "characterID","movieID","character","text"]
lines = {}
with open (lines_filepath, 'r', encoding='iso-8859-1') as f:
    for line in f:
        values = line.split("\t")
        lineObj = {}
        for i, field in enumerate(line_fields):
            lineObj[field] = values[i]
        lines[lineObj['lineID']] = lineObj

In [7]:
conv_fields = ["character1ID", "character2ID","movieID","utteranceIDs"]
conversations = []
with open (conv_filepath, 'r', encoding='iso-8859-1') as f:
    for line in f:
        values = line.split("\t")
        convObj = {}
        for i,field in enumerate(conv_fields):
            convObj[field] = values[i]        
        lineIds = eval(convObj["utteranceIDs"])
        split_lineIds = [re.findall(r'L\d+', string[0]) for string in lineIds]
        temp = []
        for string in lineIds:
            temp = re.findall(r'L\d+', string)
        
        convObj["lines"] = []
        for lineId in temp:
            convObj["lines"].append(lines[lineId])
        conversations. append(convObj)

In [25]:
qa_pairs = []
for conversation in conversations:
    for i in range(len(conversation["lines"]) -1):
        inputLine = conversation["lines"][i]["text"].replace("\r\r\n","").strip()
        targetLine = conversation["lines"][i+1]["text"].replace("\r\r\n","").strip()
        if inputLine and targetLine:
            qa_pairs.append([inputLine,targetLine])

In [26]:
datafile = os.path.join("cornell movie-dialogs corpus", "formated_movie_lines.txt")
delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

print("\nWriting newly formatted file...")
with open(datafile,'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter)
    for pair in qa_pairs:
        writer.writerow(pair)
print("Done writing file")


Writing newly formatted file...
Done writing file


In [32]:
with open(datafile, 'rb') as file:
    lines = file.readlines()
    for line in lines[:8]:
        print(line)

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell I thought we'd start with pronunciation if that's okay with you.\r\r\n"
b"Well I thought we'd start with pronunciation if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\r\n"
b"No no it's my fault -- we didn't have a proper introduction ---\tCameron.\r\r\n"
b"Cameron.\tThe thing is Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\r\n"
b"The thing is Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a date easy enough...\r\r\n"
b'Why?\tUnsolved 